# Kvinde Klinikken — AI Triage PoC
Multi-agent triage pipeline using OpenAI Agents SDK.
Classifies 53 gynecological conditions and produces structured booking requests.

In [1]:
# !pip install openai-agents pyyaml pydantic python-dotenv

In [2]:
import os
import yaml
import json
import asyncio
from datetime import date, datetime, timedelta
from typing import Optional
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

# Global model configuration — change this to switch models for all agents
MODEL = os.getenv("TRIAGE_MODEL", "gpt-4o")
print(f"Using model: {MODEL}")


Using model: gpt-5-mini


## Pydantic Models

In [ ]:
class PatientContext(BaseModel):
    """Patient information collected during intake."""
    language: str = Field(description="Detected language: da, en, or uk")
    insurance_type: str = Field(description="public, dss, or self_pay")
    has_referral: bool = Field(description="Whether patient has a GP referral")
    is_followup: bool = Field(default=False, description="Whether this is a follow-up visit")
    patient_name: Optional[str] = Field(default=None, description="Patient's full name")
    patient_age: Optional[int] = Field(default=None, description="Patient's age")
    phone_number: Optional[str] = Field(default=None, description="Patient's phone number")
    email: Optional[str] = Field(default=None, description="Patient's email address")
    doctor_preference: Optional[str] = Field(default=None, description="HS, LB, earliest, or None for standard routing")


class ConditionMatch(BaseModel):
    """Matched condition from the triage classification."""
    condition_id: int = Field(description="Condition ID (1-53) from config")
    condition_name: str = Field(description="Name of the condition")
    category: str = Field(description="A (urgent), B (high priority), or C (standard)")
    doctor: str = Field(description="Assigned doctor: HS, LB, or same_doctor")
    duration_minutes: int = Field(description="Appointment duration: 15, 20, 30, or 45")
    priority_window: Optional[str] = Field(default="standard", description="same_day, 1_2_days, 1_week, 14_days, 1_month, or standard")


class BookingRequest(BaseModel):
    """Final structured output: all info needed to book an appointment."""
    patient: PatientContext
    condition: ConditionMatch
    # Scheduling
    cycle_dependent: bool = Field(default=False)
    last_period_date: Optional[str] = Field(default=None, description="ISO date string")
    cycle_length: Optional[int] = Field(default=None, description="Cycle length in days")
    cycle_range_min: Optional[int] = Field(default=None, description="Min cycle length if irregular")
    cycle_range_max: Optional[int] = Field(default=None, description="Max cycle length if irregular")
    no_cycle: bool = Field(default=False, description="Patient has no regular cycle")
    valid_booking_window: Optional[str] = Field(default=None, description="Calculated valid date range")
    scheduling_restrictions: list[str] = Field(default_factory=list, description="e.g. not during menstruation")
    provera_recommended: bool = Field(default=False, description="Doctor may prescribe Provera to induce period")
    # Lab
    lab_required: bool = Field(default=False)
    lab_details: Optional[str] = Field(default=None)
    lab_status: Optional[str] = Field(default=None, description="completed, pending, or not_needed")
    # Documents
    questionnaire: Optional[str] = Field(default=None, description="Questionnaire to send")
    partner_questionnaire: Optional[str] = Field(default=None, description="Partner questionnaire (fertility)")
    guidance_document: Optional[str] = Field(default=None, description="Guidance document to send")
    # Booking
    tentative: bool = Field(default=False, description="True if waiting on lab results")
    booked_outside_hours: bool = Field(default=False, description="True if booked outside clinic hours")
    self_pay: bool = Field(default=False)
    self_pay_price_dkk: Optional[float] = Field(default=None)
    selected_slot: Optional[str] = Field(default=None, description="Selected appointment slot")
    notes: Optional[str] = Field(default=None, description="Special instructions")


class HandoffRequest(BaseModel):
    """Escalation to human staff."""
    patient: PatientContext
    reason: str = Field(description="Why the conversation is being escalated")
    urgency: str = Field(description="immediate, high, or normal")
    conversation_summary: str = Field(description="Summary of what was discussed")
    suggested_action: Optional[str] = Field(default=None, description="Suggested next step for staff")


In [ ]:
# Validate models compile
p = PatientContext(language="da", insurance_type="public", has_referral=True)
print("PatientContext:", p.model_dump_json(indent=2))

b = BookingRequest(
    patient=p,
    condition=ConditionMatch(condition_id=19, condition_name="IUD insertion", category="C", doctor="LB", duration_minutes=30),
)
print("\nBookingRequest:", b.model_dump_json(indent=2))


## Load Conditions Config

In [5]:
with open("conditions.yaml") as f:
    CONFIG = yaml.safe_load(f)

CONDITIONS = {c["id"]: c for c in CONFIG["conditions"]}
GROUPS = CONFIG["condition_groups"]
CYCLE_RULES = CONFIG["cycle_rules"]
QUESTIONNAIRES = CONFIG["questionnaires"]
GUIDANCE_DOCS = CONFIG["guidance_documents"]
SELF_PAY_PRICES = {p["condition_id"]: p for p in CONFIG.get("self_pay_prices", [])}

print(f"Loaded {len(CONDITIONS)} conditions, {len(GROUPS)} groups")
print(f"Cycle rules: {len(CYCLE_RULES)}, Questionnaires: {len(QUESTIONNAIRES)}")

Loaded 53 conditions, 9 groups
Cycle rules: 9, Questionnaires: 7


## Tool Functions

In [6]:
def lookup_conditions(description: str) -> str:
    """Search conditions and groups by patient description keywords. Returns JSON with matches."""
    description_lower = description.lower()
    matches = []
    group_matches = []

    # Search individual conditions by keywords
    for cond in CONFIG["conditions"]:
        for keyword in cond["keywords"]:
            if keyword.lower() in description_lower:
                matches.append({
                    "type": "condition",
                    "id": cond["id"],
                    "name": cond["name"],
                    "category": cond["category"],
                    "doctor": cond["doctor"],
                    "duration": cond["duration"],
                    "priority": cond["priority"],
                })
                break

    # Search condition groups
    for group in GROUPS:
        for keyword in group["keywords"]:
            if keyword.lower() in description_lower:
                group_matches.append({
                    "type": "group",
                    "group": group["group"],
                    "clarifying_question": group["clarifying_question"],
                    "options": group["options"],
                })
                break

    return json.dumps({"conditions": matches, "groups": group_matches}, indent=2, ensure_ascii=False)

In [7]:
# Test: should match IUD group
print("=== IUD group match ===")
print(lookup_conditions("I need help with my spiral"))
print()

# Test: should match Category A
print("=== Category A match ===")
print(lookup_conditions("I'm having heavy bleeding"))
print()

# Test: should match specific condition
print("=== Specific condition ===")
print(lookup_conditions("I need a smear test"))

=== IUD group match ===
{
  "conditions": [],
  "groups": [
    {
      "type": "group",
      "group": "IUD",
      "clarifying_question": "What do you need help with regarding your IUD (spiral)?",
      "options": [
        {
          "label": "I need a new IUD inserted",
          "condition_id": 19
        },
        {
          "label": "I need my IUD removed (strings are visible)",
          "condition_id": 20
        },
        {
          "label": "I need my IUD replaced with a new one",
          "condition_id": 21
        },
        {
          "label": "My IUD is over 8 years old or the strings cannot be found",
          "condition_id": 22
        },
        {
          "label": "I need a hysteroscopic removal or insertion (no visible strings)",
          "condition_id": 23
        }
      ]
    }
  ]
}

=== Category A match ===
{
  "conditions": [
    {
      "type": "condition",
      "id": 1,
      "name": "Acute/heavy bleeding",
      "category": "A",
      "doctor": n

In [8]:
def get_condition_details(condition_id: int) -> str:
    """Get full details for a condition by ID. Returns JSON."""
    cond = CONDITIONS.get(condition_id)
    if not cond:
        return json.dumps({"error": f"Condition {condition_id} not found"})
    return json.dumps(cond, indent=2, ensure_ascii=False)

In [9]:
# IUD insertion — should show cycle_days, lab requirement
print("=== IUD insertion (id=19) ===")
print(get_condition_details(19))

=== IUD insertion (id=19) ===
{
  "id": 19,
  "name": "IUD insertion",
  "category": "C",
  "keywords": [
    "spiral indsættelse",
    "IUD insertion",
    "ny spiral",
    "new IUD"
  ],
  "doctor": "LB",
  "duration": 30,
  "priority": "standard",
  "referral_required": true,
  "cycle_days": [
    3,
    7
  ],
  "routing_question": null,
  "lab": {
    "condition": "age_under_30",
    "test": "chlamydia",
    "description": "Negative chlamydia test required for patients under 30"
  },
  "questionnaire": null,
  "guidance": null
}


In [10]:
def calculate_cycle_window(
    last_period_date: str,
    condition_id: int,
    cycle_length: int = 28,
    cycle_range_min: int | None = None,
    cycle_range_max: int | None = None,
    no_cycle: bool = False,
) -> str:
    """Calculate valid booking window based on cycle data and condition requirements. Returns JSON."""
    cond = CONDITIONS.get(condition_id)
    if not cond or not cond.get("cycle_days"):
        return json.dumps({"cycle_dependent": False, "message": "No cycle constraint for this procedure."})

    if no_cycle:
        return json.dumps({
            "cycle_dependent": True,
            "no_cycle": True,
            "provera_recommended": True,
            "message": "Patient has no regular cycle. Doctor may prescribe Provera (10 days) to induce a period. Booking window can be calculated 2-4 days after completing the course."
        })

    cycle_days = cond["cycle_days"]

    # Special case: "just_before_next_period" (endometriosis)
    if cycle_days == "just_before_next_period":
        lp = datetime.strptime(last_period_date, "%Y-%m-%d").date()
        next_period = lp + timedelta(days=cycle_length)
        window_start = next_period - timedelta(days=3)
        window_end = next_period - timedelta(days=1)
        return json.dumps({
            "cycle_dependent": True,
            "valid_start": window_start.isoformat(),
            "valid_end": window_end.isoformat(),
            "message": f"Best scheduled just before next period: {window_start.strftime('%b %d')} - {window_end.strftime('%b %d')}"
        })

    cd_start, cd_end = cycle_days
    lp = datetime.strptime(last_period_date, "%Y-%m-%d").date()
    today = date.today()

    # Current cycle window
    window_start = lp + timedelta(days=cd_start - 1)
    window_end = lp + timedelta(days=cd_end - 1)

    if window_end < today:
        # Window passed — calculate next cycle
        if cycle_range_min and cycle_range_max:
            next_lp_earliest = lp + timedelta(days=cycle_range_min)
            next_lp_latest = lp + timedelta(days=cycle_range_max)
            next_start = next_lp_earliest + timedelta(days=cd_start - 1)
            next_end = next_lp_latest + timedelta(days=cd_end - 1)
            msg = f"This cycle's window has passed. Next window (approximate due to irregular cycle): {next_start.strftime('%b %d')} - {next_end.strftime('%b %d')}"
        else:
            next_lp = lp + timedelta(days=cycle_length)
            next_start = next_lp + timedelta(days=cd_start - 1)
            next_end = next_lp + timedelta(days=cd_end - 1)
            msg = f"This cycle's window has passed. Next window: {next_start.strftime('%b %d')} - {next_end.strftime('%b %d')}"
        return json.dumps({
            "cycle_dependent": True,
            "window_passed": True,
            "next_valid_start": next_start.isoformat(),
            "next_valid_end": next_end.isoformat(),
            "message": msg
        })

    return json.dumps({
        "cycle_dependent": True,
        "valid_start": window_start.isoformat(),
        "valid_end": window_end.isoformat(),
        "message": f"Valid booking window: {window_start.strftime('%b %d')} - {window_end.strftime('%b %d')} (cycle days {cd_start}-{cd_end})"
    })

In [11]:
# Test: IUD insertion (CD 3-7), recent period
print("=== IUD insertion, recent period ===")
print(calculate_cycle_window("2026-02-24", 19))
print()

# Test: Window already passed
print("=== Window passed ===")
print(calculate_cycle_window("2026-02-01", 19))
print()

# Test: No cycle (PCOS etc.)
print("=== No cycle ===")
print(calculate_cycle_window("2026-02-01", 19, no_cycle=True))
print()

# Test: Non-cycle-dependent (smear test)
print("=== Non-cycle-dependent ===")
print(calculate_cycle_window("2026-02-01", 38))
print()

# Test: Endometriosis "just_before_next_period"
print("=== Endometriosis special timing ===")
print(calculate_cycle_window("2026-02-15", 18, cycle_length=30))
print()

# Test: Irregular cycle range
print("=== Irregular cycle ===")
print(calculate_cycle_window("2026-02-01", 19, cycle_range_min=25, cycle_range_max=35))

=== IUD insertion, recent period ===
{"cycle_dependent": true, "valid_start": "2026-02-26", "valid_end": "2026-03-02", "message": "Valid booking window: Feb 26 - Mar 02 (cycle days 3-7)"}

=== Window passed ===
{"cycle_dependent": true, "window_passed": true, "next_valid_start": "2026-03-03", "next_valid_end": "2026-03-07", "message": "This cycle's window has passed. Next window: Mar 03 - Mar 07"}

=== No cycle ===
{"cycle_dependent": true, "no_cycle": true, "provera_recommended": true, "message": "Patient has no regular cycle. Doctor may prescribe Provera (10 days) to induce a period. Booking window can be calculated 2-4 days after completing the course."}

=== Non-cycle-dependent ===
{"cycle_dependent": false, "message": "No cycle constraint for this procedure."}

=== Endometriosis special timing ===
{"cycle_dependent": true, "valid_start": "2026-03-14", "valid_end": "2026-03-16", "message": "Best scheduled just before next period: Mar 14 - Mar 16"}

=== Irregular cycle ===
{"cycle_d

In [12]:
def get_lab_requirements(condition_id: int, patient_age: int | None = None) -> str:
    """Check if a condition requires lab work based on condition and patient age. Returns JSON."""
    cond = CONDITIONS.get(condition_id)
    if not cond or not cond.get("lab"):
        return json.dumps({"lab_required": False})

    lab = cond["lab"]
    lab_condition = lab.get("condition", "always")

    # Check age-based conditions
    if lab_condition == "age_under_30" and patient_age is not None and patient_age >= 30:
        return json.dumps({"lab_required": False, "reason": "Patient is 30 or older, lab not required."})

    if lab_condition == "age_under_45" and patient_age is not None and patient_age >= 45:
        return json.dumps({"lab_required": False, "reason": "Patient is 45 or older, lab not required."})

    return json.dumps({
        "lab_required": True,
        "test": lab.get("test") or lab.get("tests"),
        "description": lab["description"],
    }, ensure_ascii=False)

In [13]:
# IUD under 30 → chlamydia required
print("=== IUD, age 25 ===")
print(get_lab_requirements(19, 25))
print()

# IUD over 30 → not required
print("=== IUD, age 35 ===")
print(get_lab_requirements(19, 35))
print()

# Fertility → always required
print("=== Fertility ===")
print(get_lab_requirements(10, 30))
print()

# Smear test → no lab
print("=== Smear test ===")
print(get_lab_requirements(38, 40))
print()

# Menopause under 45 → blood panel
print("=== Menopause, age 42 ===")
print(get_lab_requirements(29, 42))
print()

# Menopause over 45 → not required
print("=== Menopause, age 50 ===")
print(get_lab_requirements(29, 50))

=== IUD, age 25 ===
{"lab_required": true, "test": "chlamydia", "description": "Negative chlamydia test required for patients under 30"}

=== IUD, age 35 ===
{"lab_required": false, "reason": "Patient is 30 or older, lab not required."}

=== Fertility ===
{"lab_required": true, "test": ["fertility blood panel (patient)", "fertility blood panel (partner)", "semen analysis (partner, hospital AND clinic)", "HIV, Hepatitis B & C (both, <2 years old)"], "description": "Blood tests required for both patient and partner"}

=== Smear test ===
{"lab_required": false}

=== Menopause, age 42 ===
{"lab_required": true, "test": "menopause blood panel", "description": "Menopause blood panel required for patients under 45"}

=== Menopause, age 50 ===
{"lab_required": false, "reason": "Patient is 45 or older, lab not required."}


In [14]:
def get_questionnaire(condition_id: int) -> str:
    """Get questionnaire(s) for a condition. Returns JSON."""
    result = {"questionnaires": []}
    for name, info in QUESTIONNAIRES.items():
        if condition_id in info["applies_to"]:
            entry = {"name": name}
            if info.get("target"):
                entry["target"] = info["target"]
            result["questionnaires"].append(entry)

    # Check for partner questionnaire on the condition itself
    cond = CONDITIONS.get(condition_id)
    if cond and cond.get("partner_questionnaire"):
        result["partner_questionnaire"] = cond["partner_questionnaire"]

    if not result["questionnaires"]:
        result["message"] = "No questionnaire required for this condition."
    return json.dumps(result, ensure_ascii=False)


def get_guidance_document(condition_id: int) -> str:
    """Get guidance document for a condition. Returns JSON."""
    for name, info in GUIDANCE_DOCS.items():
        if condition_id in info["applies_to"]:
            return json.dumps({"document": name})
    return json.dumps({"document": None, "message": "No guidance document for this condition."})


def get_available_slots(doctor: str, duration_minutes: int, date_range_start: str, date_range_end: str | None = None) -> str:
    """Get available appointment slots (MOCK for PoC). Returns JSON with 3 fake slots."""
    start = datetime.strptime(date_range_start, "%Y-%m-%d").date()
    slots = []
    slot_date = start
    for i in range(3):
        # Skip weekends
        while slot_date.weekday() >= 5:
            slot_date += timedelta(days=1)
        hour = 9 + (i * 2)  # 09:00, 11:00, 13:00
        slots.append({
            "date": slot_date.isoformat(),
            "time": f"{hour:02d}:00",
            "doctor": f"Dr. {doctor}",
            "duration_minutes": duration_minutes,
        })
        slot_date += timedelta(days=1)
    return json.dumps({"slots": slots})


def get_self_pay_price(condition_id: int) -> str:
    """Get self-pay price for a condition. Returns JSON."""
    price_entry = SELF_PAY_PRICES.get(condition_id)
    if price_entry:
        return json.dumps(price_entry, ensure_ascii=False)
    return json.dumps({"price_dkk": None, "message": "Price not yet available. Staff will confirm the cost."})

In [15]:
# Questionnaire for fertility (id 10) → UXOR + VIR
print("=== Fertility questionnaire ===")
print(get_questionnaire(10))
print()

# Questionnaire for incontinence (id 31) — should get 2 matches
print("=== Incontinence questionnaire ===")
print(get_questionnaire(31))
print()

# Guidance for cone biopsy (id 8)
print("=== Cone biopsy guidance ===")
print(get_guidance_document(8))
print()

# Guidance for smear test (none)
print("=== Smear test guidance ===")
print(get_guidance_document(38))
print()

# Mock slots
print("=== Mock slots ===")
print(get_available_slots("LB", 30, "2026-03-02"))
print()

# Self-pay price
print("=== Self-pay price ===")
print(get_self_pay_price(24))

=== Fertility questionnaire ===
{"questionnaires": [{"name": "Infertility Questionnaire (UXOR)"}, {"name": "Infertility Questionnaire (VIR)", "target": "partner"}], "partner_questionnaire": "Infertility Questionnaire (VIR)"}

=== Incontinence questionnaire ===
{"questionnaires": [{"name": "You & Your Gynaecological Problem"}, {"name": "Urinary Problems / Incontinence"}]}

=== Cone biopsy guidance ===
{"document": "Kegleoperation"}

=== Smear test guidance ===
{"document": null, "message": "No guidance document for this condition."}

=== Mock slots ===
{"slots": [{"date": "2026-03-02", "time": "09:00", "doctor": "Dr. LB", "duration_minutes": 30}, {"date": "2026-03-03", "time": "11:00", "doctor": "Dr. LB", "duration_minutes": 30}, {"date": "2026-03-04", "time": "13:00", "doctor": "Dr. LB", "duration_minutes": 30}]}

=== Self-pay price ===
{"condition_id": 24, "name": "Contraception counselling", "price_dkk": 1200}


## Agent Definitions

In [16]:
from agents import Agent, Runner, handoff, RunContextWrapper, SQLiteSession, function_tool
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

# Wrap existing functions as agent tools
@function_tool
def search_conditions(patient_description: str) -> str:
    """Search the clinic's condition database using the patient's description of their issue. Returns matching conditions and/or condition groups that need disambiguation."""
    return lookup_conditions(patient_description)

@function_tool  
def fetch_condition_details(condition_id: int) -> str:
    """Get full details for a specific condition including doctor, duration, priority, cycle requirements, lab requirements, and routing questions."""
    return get_condition_details(condition_id)

@function_tool
def compute_cycle_window(last_period_date: str, condition_id: int, cycle_length: int = 28, cycle_range_min: int = 0, cycle_range_max: int = 0, no_cycle: bool = False) -> str:
    """Calculate the valid booking window based on the patient's menstrual cycle and the procedure's cycle day requirements."""
    return calculate_cycle_window(
        last_period_date, condition_id, cycle_length,
        cycle_range_min if cycle_range_min > 0 else None,
        cycle_range_max if cycle_range_max > 0 else None,
        no_cycle
    )

@function_tool
def check_lab_requirements(condition_id: int, patient_age: int = 0) -> str:
    """Check if the procedure requires lab work before the appointment, considering the patient's age."""
    return get_lab_requirements(condition_id, patient_age if patient_age > 0 else None)

@function_tool
def fetch_questionnaire(condition_id: int) -> str:
    """Get the pre-visit questionnaire(s) to send to the patient for their condition."""
    return get_questionnaire(condition_id)

@function_tool
def fetch_guidance_document(condition_id: int) -> str:
    """Get the patient guidance document for a procedure, if one exists."""
    return get_guidance_document(condition_id)

@function_tool
def find_available_slots(doctor: str, duration_minutes: int, date_range_start: str, date_range_end: str = "") -> str:
    """Find available appointment slots for a specific doctor within a date range."""
    return get_available_slots(doctor, duration_minutes, date_range_start, date_range_end or None)

@function_tool
def check_self_pay_price(condition_id: int) -> str:
    """Get the self-pay price for a condition if the patient has no referral."""
    return get_self_pay_price(condition_id)

print("Agent tools ready.")

Agent tools ready.


In [ ]:
handoff_to_staff = Agent(
    name="Staff Handoff",
    model=MODEL,
    instructions="""You are transferring this patient to human clinic staff.

Read the FULL conversation and produce a HandoffRequest with:
- patient: Fill ALL PatientContext fields from conversation (language, insurance, referral, name, age, phone, email, preference). Use what was stated; leave unknown as null/default.
- reason: Clear explanation of why the patient needs human staff
- urgency: "immediate" for Category A / acute emergencies, "high" for Category B, "normal" for everything else (DSS, unclear condition, patient request)
- conversation_summary: Brief summary of what was discussed and what stage the conversation reached
- suggested_action: What the staff member should do next

Be thorough — the staff member has NOT read the chat.""",
    output_type=HandoffRequest,
)


In [ ]:
# Shared preamble for all specialist agents (Classification, Routing, Scheduling, Booking)
# Each specialist reads the full conversation, does its domain work, and hands back to Dispatch.
SPECIALIST_PREAMBLE = """BEFORE DOING ANYTHING — READ THE CONVERSATION:
1. Read the FULL conversation history from all previous agents.
2. Extract ALL information the patient has already provided.
3. Call your tools using information you ALREADY HAVE — do not re-ask.
4. Identify what is genuinely MISSING for your specific job.
5. ONLY ask the patient about missing information — one question at a time.
6. When your job is done → hand off to Dispatch immediately.

LANGUAGE — CRITICAL:
Check the patient's messages. Respond in the SAME language they write in (English, Danish, or Ukrainian). NEVER switch languages mid-conversation.

ESCAPE HATCH:
If the patient says "speak to staff" / "talk to a person" / "I want a human" → hand off to Dispatch immediately without doing anything else."""


booking_agent = Agent(
    name="Booking",
    model=MODEL,
    handoff_description="Finalizes appointment booking — presents slots, collects confirmation, produces BookingRequest.",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
You are the booking agent for Kvinde Klinikken, a Danish gynecology clinic. You finalize the appointment.

{SPECIALIST_PREAMBLE}

YOUR JOB:
1. Read the FULL conversation — extract: patient info, condition, doctor, duration, cycle window, lab status
2. Call fetch_questionnaire(condition_id) — share questionnaire info with patient if applicable
3. Call fetch_guidance_document(condition_id) — share guidance info with patient if applicable
4. If self-pay → call check_self_pay_price(condition_id)
5. Call find_available_slots(doctor, duration_minutes, date_range_start, date_range_end)
6. Present 2-3 slot options naturally to the patient as a text message
7. STOP and WAIT for the patient to respond with their choice

CRITICAL — TWO-STEP BOOKING:
Step A: Present the available slots to the patient as a text response. You MUST produce a text message — do NOT produce a BookingRequest yet. Wait for the patient's reply.
Step B: After the patient confirms a specific slot → THEN produce the BookingRequest with ALL fields filled.

Do NOT combine Step A and Step B in one turn. Present slots first, wait for confirmation, then produce the structured output.

FILL EVERY FIELD from conversation (do NOT invent values):
- patient: language, insurance, referral, name, age (only if stated), phone, email, preference
- condition: id, name, category, doctor, duration, priority
- scheduling: cycle_dependent, last_period_date, cycle_length, no_cycle, valid_booking_window, provera_recommended
- lab: lab_required, lab_details, lab_status
- documents: questionnaire, partner_questionnaire, guidance_document
- booking: self_pay, price, selected_slot, tentative, booked_outside_hours

If a field was never mentioned (like age), leave it as null — do NOT guess or hallucinate values.
""",
    tools=[fetch_questionnaire, fetch_guidance_document, find_available_slots, check_self_pay_price],
    output_type=BookingRequest,
)


In [19]:
scheduling_agent = Agent(
    name="Scheduling",
    model=MODEL,
    handoff_description="Handles cycle timing and lab prerequisites before booking.",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
You are the scheduling agent for Kvinde Klinikken, a Danish gynecology clinic. You handle cycle timing and lab prerequisites.

{SPECIALIST_PREAMBLE}

YOUR JOB:
1. Read the conversation — find: condition_id, patient age, any cycle info, lab mentions
2. ALWAYS call check_lab_requirements(condition_id, patient_age) — use age from conversation
3. Check if the procedure is cycle-dependent (from condition details in conversation)
4. For cycle-dependent procedures:
   a. Patient already gave last period + cycle length → call compute_cycle_window() directly
   b. Patient mentioned no periods / PCOS / amenorrhea → call compute_cycle_window() with no_cycle=true
   c. Cycle info genuinely missing → ask ONE question at a time
5. For lab: if required and patient hasn't mentioned results → inform them once
6. When done → hand off to Dispatch

TOOL USAGE — MANDATORY:
- You MUST call check_lab_requirements() for EVERY patient
- If condition has cycle_days, you MUST call compute_cycle_window()
- If patient has no periods/amenorrhea → call with no_cycle=true for Provera recommendation
- Do NOT guess scheduling info — always use the tools
""",
    tools=[compute_cycle_window, check_lab_requirements],
)


In [20]:
routing_agent = Agent(
    name="Routing",
    model=MODEL,
    handoff_description="Determines which doctor (HS or LB) should see the patient based on condition and routing rules.",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
You are the routing agent for Kvinde Klinikken, a Danish gynecology clinic. You determine which doctor should see the patient.

{SPECIALIST_PREAMBLE}

YOUR JOB:
1. Read the conversation — find the condition_id and category
2. Call fetch_condition_details() with the condition_id
3. Check: does this condition have a routing_question? Is the answer already in conversation?
   - Patient age → often already stated
   - IUD string visibility → may need asking
   - Previous doctor seen → may already be mentioned
   - Second opinion type → may already be clear
4. If answer is known → determine doctor → hand off to Dispatch
5. If genuinely missing → ask ONE natural question, then determine doctor → hand off to Dispatch

ROUTING RULES:
- Condition 15 (premenopausal bleeding): age >45 → Dr. HS; age ≤45 → Dr. LB
- Conditions 20, 21 (IUD removal/replacement): strings not visible → Dr. HS; visible → Dr. LB
- Condition 29 (menopause new): seen Dr. Skensved before → Dr. HS; no + uncomplicated → Dr. LB
- Condition 30 (menopause follow-up): route to same doctor as last visit
- Condition 52 (second opinion): fertility-related → Dr. LB; other → Dr. HS
- Doctor preference "earliest" → both doctors acceptable
- Doctor preference "HS" or "LB" → override standard routing
""",
    tools=[fetch_condition_details],
)


In [21]:
classification_agent = Agent(
    name="Classification",
    model=MODEL,
    handoff_description="Identifies the patient's gynecological condition from their description.",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
You are the classification agent for Kvinde Klinikken, a Danish gynecology clinic. You identify the patient's condition.

{SPECIALIST_PREAMBLE}

YOUR JOB:
1. Read the conversation — has the patient already described their issue?
2. If the condition is clear → call search_conditions() to confirm → hand off to Dispatch
3. If the condition is vague or matches a CONDITION GROUP → ask ONE natural follow-up to narrow it down
4. If NO condition was mentioned at all → ask: "Could you tell me a bit about what brings you in? For example, is it related to bleeding, pain, a check-up, contraception, fertility, or something else?"

CATEGORY A — URGENT:
If the condition is Category A (heavy/acute bleeding, sudden severe pain, suspected ectopic pregnancy, abortion request):
→ Tell the patient empathetically that this needs urgent attention
→ Hand off to Dispatch (Dispatch handles the staff escalation)

RULES:
- NEVER present numbered option lists — use natural conversation
- ONE question at a time
- Do NOT re-ask what the patient already told intake
- Use the patient's own words to confirm understanding
""",
    tools=[search_conditions],
)


In [ ]:
intake_agent = Agent(
    name="Intake",
    model=MODEL,
    handoff_description="First point of contact — collects insurance, referral, name, phone, doctor preference.",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
You are the intake agent for Kvinde Klinikken, a Danish gynecology clinic. You are the first point of contact.

LANGUAGE — CRITICAL:
Detect language from the WORDS the patient uses (not from names or context):
- English words ("Hi", "Hello", "I have", "I need") → respond in English
- Danish words ("Hej", "Jeg har", "Jeg skal") → respond in Danish
- Ukrainian → respond in Ukrainian
NEVER switch languages mid-conversation.

YOUR JOB — COLLECT MISSING INFO:
Read the patient's messages. Extract everything already stated.
Required information (only ask for what is MISSING):
- Insurance type (public / DSS / self-pay)
- Referral status (yes / no)
- Patient name
- Phone number
- Doctor preference (Dr. HS / Dr. LB / earliest available / no preference)

DO NOT ask for age — later agents handle that if needed.
Email is optional — only ask if nothing else to collect.

In ONE message: acknowledge what you already know, ask for the FIRST missing item.

If DSS/private insurance is mentioned → hand off to Dispatch (it handles staff escalation).
If no referral → offer self-pay path: "Without a referral, the consultation would be self-pay. Would you like to proceed?"

WHEN DONE:
Once you have: insurance ✓, referral ✓, name ✓, phone ✓, preference ✓
→ Hand off to Dispatch.

ESCAPE HATCH:
If patient says "speak to staff" / "talk to a person" → hand off to Dispatch immediately.
""",
)


In [ ]:
# === DISPATCH AGENT ===
# Central controller — manages the flow between specialist agents.
# Specialists hand back to Dispatch when done; Dispatch routes to the next one.
# Only Dispatch produces patient-facing messages for staff escalation.
# Note: GDPR consent is handled programmatically BEFORE the pipeline starts — agents don't manage it.

dispatch_agent = Agent(
    name="Dispatch",
    model=MODEL,
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
You are the dispatch agent for Kvinde Klinikken's triage pipeline. You manage the flow between specialist agents.

You do NOT interact with the patient directly. You read the conversation state and immediately hand off to the right specialist agent. The ONLY exception is staff escalation (see below), where you give one brief reassuring message before handing off.

LANGUAGE — CRITICAL:
If you produce any text (staff escalation), use the same language the patient has been writing in.

=== CHECK FOR STAFF ESCALATION FIRST ===
If ANY of these are true:
- Patient said "speak to staff" / "talk to a person" / "I want a human"
- Patient has DSS or private insurance (confirmed in conversation)
- A Category A (urgent) condition was identified (acute bleeding, severe pain, ectopic pregnancy, abortion)
→ Tell the patient (in their language): "The clinic has your information and a staff member will reach out to you shortly."
→ Hand off to Staff Handoff.

=== NORMAL FLOW — CHECK IN ORDER ===
Read the conversation and determine the EARLIEST incomplete step:

1. INTAKE NOT COMPLETE
   Check: name, phone, insurance type, referral status, doctor preference — all must be present in the conversation.
   If any are missing → hand off to Intake.

2. NO CONDITION IDENTIFIED
   Patient's medical issue has not been classified yet.
   → Hand off to Classification.

3. CONDITION IDENTIFIED BUT NO DOCTOR DETERMINED
   Condition found but doctor routing not yet done.
   → Hand off to Routing.

4. DOCTOR DETERMINED BUT SCHEDULING NOT DONE
   Doctor assigned but cycle/lab not yet handled.
   → Hand off to Scheduling.

5. SCHEDULING COMPLETE → READY TO BOOK
   Lab and cycle handled, ready for appointment booking.
   → Hand off to Booking.

=== RULES ===
- Hand off IMMEDIATELY after determining the state. Do not delay.
- Do not ask the patient anything (except the staff escalation message).
- Do not repeat or summarize what previous agents said.
- If unsure which step is incomplete, hand off to the EARLIEST agent in the pipeline.
- NEVER produce a BookingRequest or HandoffRequest yourself — terminal agents do that.
""",
    handoffs=[intake_agent, classification_agent, routing_agent, scheduling_agent, booking_agent, handoff_to_staff],
)


# Wire all specialist agents to hand back to Dispatch
intake_agent.handoffs = [dispatch_agent]
classification_agent.handoffs = [dispatch_agent]
routing_agent.handoffs = [dispatch_agent]
scheduling_agent.handoffs = [dispatch_agent]
booking_agent.handoffs = [dispatch_agent]
# Staff Handoff has no handoffs — it's terminal (produces HandoffRequest)


# === Verify pipeline structure ===
print("=== Dispatch Pipeline ===")
print(f"  Dispatch → routes to: {[a.name for a in [intake_agent, classification_agent, routing_agent, scheduling_agent, booking_agent, handoff_to_staff]]}")
print()
for agent in [intake_agent, classification_agent, routing_agent, scheduling_agent, booking_agent]:
    targets = [h.agent.name if hasattr(h, 'agent') else (h.name if hasattr(h, 'name') else str(h)) for h in agent.handoffs]
    print(f"  {agent.name} → hands back to: {targets}")
print()
print(f"  Staff Handoff → terminal (output: {handoff_to_staff.output_type.__name__})")
print(f"  Booking → terminal (output: {booking_agent.output_type.__name__})")
print()
print("Tools per agent:")
for agent in [intake_agent, classification_agent, routing_agent, scheduling_agent, booking_agent]:
    tool_names = [t.name for t in agent.tools] if agent.tools else []
    print(f"  {agent.name}: {tool_names or 'none'}")


## Test Helper
Multi-turn test runner with agent tracking. GDPR consent is handled programmatically before the pipeline.

In [ ]:
async def run_test(test_name, initial_message, follow_ups=None):
    """Run a multi-turn triage test with agent tracking.
    
    Starts with the Dispatch agent, which routes to the appropriate specialist.
    GDPR consent is assumed to be handled programmatically before the pipeline.
    """
    session = SQLiteSession(test_name, "triage_conversations.db")
    agent = dispatch_agent  # Always start with Dispatch
    all_messages = [initial_message] + (follow_ups or [])
    
    for turn, msg in enumerate(all_messages, 1):
        # Print patient message
        patient_preview = msg if len(msg) <= 120 else msg[:120] + "..."
        print(f"  Patient [{turn}]: {patient_preview}")
        
        result = await Runner.run(agent, msg, session=session)
        agent = result.last_agent  # track pipeline progression
        
        if not isinstance(result.final_output, str):
            output_type = type(result.final_output).__name__
            print(f"  Agent   [{turn}] ({agent.name}) → {output_type}:")
            print(result.final_output.model_dump_json(indent=2))
            return result
        else:
            text = result.final_output.strip()
            if not text:
                print(f"  Agent   [{turn}] ({agent.name}) → [handed off silently]")
            elif len(text) > 300:
                print(f"  Agent   [{turn}] ({agent.name}): {text[:300]}...")
            else:
                print(f"  Agent   [{turn}] ({agent.name}): {text}")
        print()  # blank line between turns

    print(f"  --- Ended after {len(all_messages)} turns — last agent: {agent.name} ---")
    return result


def print_summary(result):
    """Print a concise test result summary."""
    output_type = type(result.final_output).__name__
    agent_name = result.last_agent.name
    if hasattr(result.final_output, 'model_dump_json'):
        print(f"\n>> RESULT: {output_type} from [{agent_name}]")
        data = result.final_output.model_dump()
        if output_type == "HandoffRequest":
            p = data.get('patient', {})
            print(f"   Language: {p.get('language')}")
            print(f"   Reason: {data.get('reason', '')[:120]}")
            print(f"   Urgency: {data.get('urgency')}")
        elif output_type == "BookingRequest":
            p = data.get('patient', {})
            c = data.get('condition', {})
            print(f"   Language: {p.get('language')}")
            print(f"   Patient: {p.get('patient_name')} | Phone: {p.get('phone_number')} | Email: {p.get('email')}")
            print(f"   Insurance: {p.get('insurance_type')} | Referral: {p.get('has_referral')} | Self-pay: {data.get('self_pay')}")
            print(f"   Condition: [{c.get('condition_id')}] {c.get('condition_name')} (Cat {c.get('category')})")
            print(f"   Doctor: {c.get('doctor')} | Duration: {c.get('duration_minutes')}min | Priority: {c.get('priority_window')}")
            print(f"   Preference: {p.get('doctor_preference')}")
            if data.get('cycle_dependent'):
                print(f"   Cycle: dependent | Window: {data.get('valid_booking_window')} | No-cycle: {data.get('no_cycle')} | Provera: {data.get('provera_recommended')}")
            if data.get('lab_required'):
                print(f"   Lab: {data.get('lab_details', '')[:100]} | Status: {data.get('lab_status')}")
            if data.get('questionnaire') or data.get('partner_questionnaire'):
                print(f"   Questionnaire: {data.get('questionnaire')} | Partner: {data.get('partner_questionnaire')}")
            if data.get('guidance_document'):
                print(f"   Guidance: {data.get('guidance_document')}")
            if data.get('self_pay_price_dkk'):
                print(f"   Price: {data.get('self_pay_price_dkk')} DKK")
    else:
        print(f"\n>> RESULT: Text response from [{agent_name}]")
        print(f"   {result.final_output[:300]}")


## Test 1: Category C — IUD Insertion

In [25]:
print("Test 1: Category C — IUD Insertion")
print("Expected: Full pipeline → BookingRequest (IUD insertion, LB, cycle days 3-7)")
print("=" * 60)

result = await run_test(
    "test_1_iud",
    # Patient provides most info upfront
    "Hi, my name is Maria Hansen, email maria@test.dk, phone 12345678. "
    "I have a referral and public health insurance. I need to have a new IUD inserted. "
    "I am 25 years old. No doctor preference.",
    follow_ups=[
        # After consent, agent may ask for remaining info or hand off
        "Yes, I\'d like the earliest available please.",
        # Classification may ask about the IUD
        "A brand new IUD insertion, this is my first one.",
        # Scheduling may ask about cycle
        "My last period started on February 20th.",
        "My cycle is regular, about 28 days.",
        # Scheduling may ask about lab (chlamydia for under 30)
        "I don\'t have a recent chlamydia test.",
        # Booking may present slots
        "The first available slot works for me.",
        "Yes, that\'s perfect.",
    ]
)
print_summary(result)


Test 1: Category C — IUD Insertion
Expected: Full pipeline → BookingRequest (IUD insertion, LB, cycle days 3-7)
  Patient [1]: Hi, my name is Maria Hansen, email maria@test.dk, phone 12345678. I have a referral and public health insurance. I need ...
  Agent   [1] (Scheduling) → [handed off silently]

  Patient [2]: Yes, I consent.
  Agent   [2] (Dispatch): Thanks, Maria — I have everything I need.

- Next suitable booking window for insertion (cycle days 3–7) is Mar 22–26, 2026 (this cycle’s window has passed).
- A negative chlamydia test is required (you’re 25) and you don’t have a recent test.

The clinic has your information and a staff member will...

  Patient [3]: Yes, I'd like the earliest available please.
  Agent   [3] (Scheduling): Jeg sender nu dine oplysninger videre til bookingsafdelingen, så de kan finde den tidligst mulige tid i det næste gyldige vindue (ca. 22.–26. marts) og give instruktioner om klamydiatest. En medarbejder kontakter dig snarest.

  Patient [4]: A bra

CancelledError: 

## Test 2: Category A — Urgent Handoff (heavy bleeding)

In [26]:
print("Test 2: Category A — Urgent Handoff (heavy bleeding)")
print("Expected: Classification → HandoffRequest (immediate urgency)")
print("=" * 60)

result = await run_test(
    "test_2_urgent",
    "Hi, I have very heavy bleeding and severe pain. I have public "
    "health insurance and a referral. My name is Anna Larsen, phone 23456789.",
    follow_ups=[
        # After consent, intake processes info and hands to classification
        # Classification should recognize Category A and hand off immediately
        "No preference for doctor.",
        "The bleeding started yesterday and it\'s getting worse.",
    ]
)
print_summary(result)


Test 2: Category A — Urgent Handoff (heavy bleeding)
Expected: Classification → HandoffRequest (immediate urgency)
  Patient [1]: Hi, I have very heavy bleeding and severe pain. I have public health insurance and a referral. My name is Anna Larsen, p...
  Agent   [1] (Staff Handoff) → HandoffRequest:
{
  "patient": {
    "language": "en",
    "gdpr_consent": true,
    "insurance_type": "public",
    "has_referral": true,
    "is_followup": false,
    "patient_name": "Anna Larsen",
    "patient_age": null,
    "phone_number": "23456789",
    "email": null,
    "doctor_preference": "None"
  },
  "reason": "Acute heavy vaginal bleeding with severe pelvic pain that began yesterday and is worsening.",
  "urgency": "immediate",
  "conversation_summary": "Patient Anna Larsen reports very heavy vaginal bleeding and severe pelvic pain starting yesterday and worsening. She has public health insurance and a GP referral, consents to GDPR processing, provided phone number 23456789, and has no docto

 We didnt get consent here

## Test 3: DSS Insurance — Immediate Handoff

In [ ]:
print("Test 3: DSS Insurance — Immediate Handoff")
print("Expected: Dispatch detects DSS → HandoffRequest")
print("=" * 60)

result = await run_test(
    "test_3_dss",
    "Hi, I have private insurance through Dansk Sundhedssikring. I need to book "
    "an appointment for a fertility consultation.",
    follow_ups=[]  # Dispatch should detect DSS and hand off immediately
)
print_summary(result)


no explicit consent 

## Test 4: Self-Pay Patient — No Referral

In [27]:
print("Test 4: Self-Pay Patient — No Referral")
print("Expected: Pipeline handles self-pay path → BookingRequest or progresses")
print("=" * 60)

result = await run_test(
    "test_4_selfpay",
    "Hello, I don\'t have a referral but I\'d like to book as a self-paying patient. "
    "I need contraception counselling. My name is Lisa Berg, lisa@email.com, phone 34567890. "
    "I have public health insurance. No doctor preference.",
    follow_ups=[
        # Classification should match contraception counselling
        "I\'d like to discuss my options for contraception.",
        # Routing / Scheduling
        "No, I\'m not currently on any contraception.",
        # Booking
        "The earliest slot please.",
        "Yes, that works.",
    ]
)
print_summary(result)


Test 4: Self-Pay Patient — No Referral
Expected: Pipeline handles self-pay path → BookingRequest or progresses
  Patient [1]: Hello, I don't have a referral but I'd like to book as a self-paying patient. I need contraception counselling. My name ...
  Agent   [1] (Booking) → BookingRequest:
{
  "patient": {
    "language": "en",
    "gdpr_consent": true,
    "insurance_type": "public",
    "has_referral": false,
    "is_followup": false,
    "patient_name": "Lisa Berg",
    "patient_age": 32,
    "phone_number": "34567890",
    "email": "lisa@email.com",
    "doctor_preference": "None"
  },
  "condition": {
    "condition_id": 24,
    "condition_name": "Contraception counselling",
    "category": "C",
    "doctor": "LB",
    "duration_minutes": 30,
    "priority_window": "standard"
  },
  "cycle_dependent": false,
  "last_period_date": null,
  "cycle_length": null,
  "cycle_range_min": null,
  "cycle_range_max": null,
  "no_cycle": false,
  "valid_booking_window": null,
  "scheduling_r

## Test 5: Irregular Cycle — PCOS with No Periods

In [28]:
print("Test 5: Irregular Cycle — PCOS with No Periods")
print("Expected: Scheduling handles no-cycle → Provera recommendation")
print("=" * 60)

result = await run_test(
    "test_5_pcos",
    "Hi, my name is Sofie Nielsen, sofie@test.dk, phone 45678901. "
    "Public health insurance, I have a referral. I have PCOS and have not had a "
    "period for several months. I need a new evaluation. I am 28 years old. "
    "No doctor preference.",
    follow_ups=[
        # Classification should match PCOS
        "Yes, I was diagnosed with PCOS and need a follow-up evaluation.",
        # Scheduling may ask about cycle
        "I haven\'t had a period in about 6 months.",
        "No, my cycles are very irregular, sometimes I skip months entirely.",
        # Lab / Booking
        "No, I don\'t have any recent test results.",
        "The first available appointment please.",
        "Yes, that\'s fine.",
    ]
)
print_summary(result)


Test 5: Irregular Cycle — PCOS with No Periods
Expected: Scheduling handles no-cycle → Provera recommendation
  Patient [1]: Hi, my name is Sofie Nielsen, sofie@test.dk, phone 45678901. Public health insurance, I have a referral. I have PCOS and...
  Agent   [1] (Booking) → BookingRequest:
{
  "patient": {
    "language": "en",
    "gdpr_consent": true,
    "insurance_type": "public",
    "has_referral": true,
    "is_followup": true,
    "patient_name": "Sofie Nielsen",
    "patient_age": 28,
    "phone_number": "45678901",
    "email": "sofie@test.dk",
    "doctor_preference": "None"
  },
  "condition": {
    "condition_id": 40,
    "condition_name": "PCOS — follow-up",
    "category": "C",
    "doctor": "LB",
    "duration_minutes": 15,
    "priority_window": "standard"
  },
  "cycle_dependent": false,
  "last_period_date": null,
  "cycle_length": null,
  "cycle_range_min": null,
  "cycle_range_max": null,
  "no_cycle": true,
  "valid_booking_window": null,
  "scheduling_restriction

## Test 6: Interactive Multi-Turn Conversation

In [29]:
async def interactive_triage():
    """Interactive triage loop — type messages, agents guide you through. Type 'quit' to exit."""
    session = SQLiteSession("triage_interactive", "triage_conversations.db")
    current_agent = dispatch_agent  # Start with Dispatch — it routes to the right specialist
    max_turns = 25

    print("Kvinde Klinikken AI Triage (type 'quit' to exit)")
    print("=" * 50)

    for turn in range(1, max_turns + 1):
        user_input = input(f"\nPatient (turn {turn}): ")
        if user_input.lower() in ("quit", "exit"):
            print("Session ended.")
            return None

        result = await Runner.run(current_agent, user_input, session=session)
        current_agent = result.last_agent  # resume from where the pipeline left off

        if not isinstance(result.final_output, str):
            output_type = type(result.final_output).__name__
            print(f"\n[{result.last_agent.name}] produced {output_type}:")
            print(result.final_output.model_dump_json(indent=2))
            return result.final_output
        else:
            print(f"\n[{result.last_agent.name}]: {result.final_output}")

    print(f"\nReached max turns ({max_turns}). Session ended.")
    return None

final = await interactive_triage()

if final:
    print(f"Type: {type(final).__name__}")
    print(final.model_dump_json(indent=2))


Kvinde Klinikken AI Triage (type 'quit' to exit)

[Classification]: 


CancelledError: 

## Test 7: Doctor Preference — Earliest Available

In [31]:
print("Test 7: Doctor Preference — Earliest Available")
print("Expected: doctor_preference=\'earliest\', pipeline progresses")
print("=" * 60)

result = await run_test(
    "test_7_earliest",
    "Hi, my name is Mette Olsen, mette@test.dk, phone 56789012. "
    "Public health insurance, I have a referral. I would like the earliest "
    "available appointment. I need to have a hysteroscopy done. I am 40 years old.",
    follow_ups=[
        # Classification should match hysteroscopy
        "It\'s a diagnostic hysteroscopy my doctor referred me for.",
        # Scheduling — cycle dependent (CD 4-8)
        "My last period started on February 18th.",
        "About 28 days, pretty regular.",
        # Lab
        "No special test results needed that I know of.",
        # Booking
        "The earliest slot please.",
        "Yes, that works perfectly.",
    ]
)
print_summary(result)


## Test 8: Premenopausal Bleeding — Age >45 Routes to HS

In [ ]:
print("Test 8: Premenopausal Bleeding — Age >45 Routes to HS")
print("Expected: Routing asks age → routes to Dr. HS")
print("=" * 60)

result = await run_test(
    "test_8_premenopausal",
    "Hi, my name is Birgitte Madsen, birgitte@test.dk, phone 67890123. "
    "Public health insurance and I have a referral. I have irregular bleeding. "
    "I am 48 years old. No doctor preference.",
    follow_ups=[
        # Classification should match irregular/premenopausal bleeding
        "The bleeding is irregular and heavier than usual.",
        # Routing should ask about age or note 48 → route to HS
        "Yes, I\'m 48.",
        # Scheduling
        "My last period was about 2 weeks ago but it\'s been unpredictable.",
        "Anywhere from 21 to 35 days lately.",
        # Booking
        "The first available slot please.",
        "Yes, that\'s fine.",
    ]
)
print_summary(result)


## Test 9: IUD Disambiguation — Vague "spiral" Input

In [ ]:
print("Test 9: IUD Disambiguation — Vague 'spiral' Input")
print("Expected: Classification asks follow-up about IUD type → specific condition")
print("=" * 60)

result = await run_test(
    "test_9_iud_vague",
    "Hi, my name is Karen Holm, karen@test.dk, phone 78901234. "
    "Public health insurance, I have a referral. I need help with my IUD. "
    "No doctor preference.",
    follow_ups=[
        # Classification should ask: new insertion, removal, replacement?
        "I need to have it removed. The strings are visible.",
        # Routing → LB for standard removal
        # Scheduling — cycle dependent?
        "My last period started on February 22nd.",
        "About 28 days.",
        # Booking
        "The first available slot please.",
        "Yes, that works.",
    ]
)
print_summary(result)


## Test 10: English Language — Full Flow

In [ ]:
print("Test 10: English Language — Full Flow")
print("Expected: Full pipeline in English → BookingRequest")
print("=" * 60)

result = await run_test(
    "test_10_english",
    "Hello, my name is Emily Brown, emily@mail.com, phone 89012345. "
    "I have public health insurance and a referral from my GP. "
    "I\'ve been having pelvic pain during intercourse. "
    "It\'s been going on for a few weeks. I\'m 32 years old. No doctor preference.",
    follow_ups=[
        # Classification should match dyspareunia / pelvic pain
        "The pain is during intercourse, deep inside.",
        # Routing
        "No, this is the first time I\'m seeing a specialist for this.",
        # Scheduling
        "My last period was February 19th, cycle is about 30 days.",
        # Booking
        "The first available slot please.",
        "Yes, perfect.",
    ]
)
print_summary(result)


## Test 11: Escape Hatch — "Speak to Staff"

In [ ]:
print("Test 11: Escape Hatch — 'Speak to Staff'")
print("Expected: HandoffRequest produced when patient asks for staff")
print("=" * 60)

session_escape = SQLiteSession("test_11_escape", "triage_conversations.db")
agent = dispatch_agent  # Start with Dispatch

# Turn 1: normal start
result = await Runner.run(agent, "Hello, I have public health insurance and a referral.", session=session_escape)
agent = result.last_agent
print(f"  Turn 1 [{agent.name}]: {result.final_output[:200]}")

# Turn 2: provide name/phone
result = await Runner.run(agent, "My name is Test Patient, phone 11112222. No doctor preference.", session=session_escape)
agent = result.last_agent
print(f"  Turn 2 [{agent.name}]: {result.final_output[:200] if isinstance(result.final_output, str) else type(result.final_output).__name__}")

# Turn 3: patient requests human
result = await Runner.run(agent, "Actually, I would like to speak to a real person instead please.", session=session_escape)
agent = result.last_agent
print(f"  Turn 3 [{agent.name}]:")
if hasattr(result.final_output, 'model_dump_json'):
    print(f"  → {type(result.final_output).__name__}")
    print(result.final_output.model_dump_json(indent=2))
else:
    print(f"  {result.final_output}")


## Test 12: Abortion — Emergency, No Referral Needed

In [ ]:
print("Test 12: Abortion — Emergency, No Referral Needed")
print("Expected: Category A → HandoffRequest (immediate urgency)")
print("=" * 60)

result = await run_test(
    "test_12_abortion",
    "Hi, I need to book an abortion. I don\'t have a referral. "
    "My name is Anne Jensen, anne@test.dk, phone 90123456. I have public insurance.",
    follow_ups=[
        # After consent + intake, classification should recognize Category A
        "I just found out I\'m pregnant and I need an abortion as soon as possible.",
    ]
)
print_summary(result)


## Test 13: Endometriosis — "Just Before Next Period" Timing

In [ ]:
print("Test 13: Endometriosis — 'Just Before Next Period' Timing")
print("Expected: Scheduling calculates 'just_before_next_period' window")
print("=" * 60)

result = await run_test(
    "test_13_endometriosis",
    "Hi, my name is Camilla Poulsen, camilla@test.dk, phone 01234567. "
    "Public health insurance, I have a referral. I have been referred for endometriosis. "
    "I am 29 years old. No doctor preference.",
    follow_ups=[
        "Yes, I\'ve been diagnosed with endometriosis and need further evaluation.",
        "My last period started on February 15th and my cycle is about 30 days.",
        "Yes, that timing works for me.",
        "The first available slot please.",
        "Yes, perfect.",
    ]
)
print_summary(result)


## Test 14: Fertility — Partner Labs & Dual Questionnaire

In [ ]:
print("Test 14: Fertility — Partner Labs & Dual Questionnaire")
print("Expected: Lab requirements for both partners, UXOR + VIR questionnaires")
print("=" * 60)

result = await run_test(
    "test_14_fertility",
    "Hi, my name is Louise Berg, louise@test.dk, phone 11223344. "
    "Public health insurance, I have a referral. We have been trying to get pregnant "
    "for over a year and would like a fertility evaluation. I am 33 years old. "
    "No doctor preference.",
    follow_ups=[
        "Yes, we\'ve been trying for about 14 months now.",
        "My cycle is quite regular, about 28 days. Last period February 20th.",
        "No, we haven\'t had any fertility tests done yet.",
        "The first available slot please.",
        "Yes, that works.",
    ]
)
print_summary(result)


## Test 15: Menopause Follow-Up — Route to Same Doctor

In [ ]:
print("Test 15: Menopause Follow-Up — Route to Same Doctor")
print("Expected: Routes to same doctor (Dr. HS / Dr. Skensved)")
print("=" * 60)

result = await run_test(
    "test_15_menopause",
    "Hi, my name is Kirsten Sorensen, kirsten@test.dk, phone 22334455. "
    "Public health insurance, I am an existing patient. I need a follow-up "
    "for my menopause treatment. I last saw Dr. Skensved.",
    follow_ups=[
        "Yes, this is a follow-up appointment.",
        "I saw Dr. Skensved about 6 months ago.",
        "The first available slot with Dr. Skensved please.",
        "Yes, that works.",
    ]
)
print_summary(result)


## Test 16: Second Opinion — Fertility (LB) vs Non-Fertility (HS)

In [ ]:
print("Test 16: Second Opinion — Fertility (LB) vs Non-Fertility (HS)")
print("Expected: Fertility → LB, Non-fertility → HS")
print("=" * 60)

# Fertility second opinion → should route to LB
print("\n--- Fertility second opinion ---")
result_fertility = await run_test(
    "test_16a_fertility_opinion",
    "Hi, I\'m Sarah Thompson, sarah@test.dk, phone 33445566. "
    "Public insurance, I have a referral. I\'d like a second opinion about my "
    "fertility treatment from another clinic. I\'m 35. No doctor preference.",
    follow_ups=[
        "Yes, it\'s specifically about fertility treatment I\'ve been getting.",
        "The first available slot please.",
        "Yes, that works.",
    ]
)
print_summary(result_fertility)

# Non-fertility second opinion → should route to HS
print("\n--- Non-fertility second opinion ---")
result_other = await run_test(
    "test_16b_other_opinion",
    "Hi, my name is Lene Kristensen, lene@test.dk, phone 44556677. "
    "Public health insurance, I have a referral. I would like a second opinion "
    "about my endometriosis diagnosis. I am 38 years old. No doctor preference.",
    follow_ups=[
        "No, it\'s not about fertility. I want a second opinion on my endometriosis.",
        "The first available slot please.",
        "Yes, that works.",
    ]
)
print_summary(result_other)


## Test 17: Category B — Cancer Package (1 Week Priority)

In [ ]:
print("Test 17: Category B — Cancer Package (1 Week Priority)")
print("Expected: Category B, priority 1_week, HandoffRequest or BookingRequest")
print("=" * 60)

result = await run_test(
    "test_17_cancer",
    "Hi, my name is Helle Thomsen, helle@test.dk, phone 55667788. "
    "Public health insurance, I have a referral from my doctor as a cancer package. "
    "I am 55 years old. No doctor preference.",
    follow_ups=[
        "Yes, my doctor referred me under the cancer package for further investigation.",
        "No, I have no preference for doctor.",
        "The first available slot please.",
        "Yes, that works.",
    ]
)
print_summary(result)


## Test 18: Lichen Sclerosus — Group Disambiguation

In [ ]:
print("Test 18: Lichen Sclerosus — Group Disambiguation")
print("Expected: Classification asks if new, follow-up, or annual check")
print("=" * 60)

result = await run_test(
    "test_18_lichen",
    "Hi, my name is Inge Petersen, inge@test.dk, phone 66778899. "
    "Public health insurance, I have a referral. I have lichen sclerosus. "
    "No doctor preference.",
    follow_ups=[
        "This is a new referral, I haven\'t been seen for this before.",
        "The first available slot please.",
        "Yes, that\'s fine.",
    ]
)
print_summary(result)
